In [ ]:
from google.colab import userdata
gemini_api_key = userdata.get('gemini_api_key')


In [ ]:
import requests
import json
from pprint import pp


In [ ]:
# 使用 HTTP 請求直接呼叫 Gemini API
payload = {
    "contents": [
        {
            "parts": [
                {
                    "text": "你好，最近過得如何?"
                }
            ]
        }
    ],
    "generationConfig": {
        "temperature": 0  # 可以改改看 溫度
    }
}

headers = {
    "Content-Type": "application/json"
}

# Gemini API endpoint
url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?key={gemini_api_key}"

response = requests.post(url, headers=headers, data=json.dumps(payload))

obj = json.loads(response.text)

pp(obj)


In [ ]:
%pip install google-generativeai


In [ ]:
import google.generativeai as genai

# 設定 API Key
genai.configure(api_key=gemini_api_key)

# 初始化模型
model = genai.GenerativeModel('gemini-1.5-flash')

# 生成回應
response = model.generate_content("你好，最近過得如何?")

print(response.text)


In [ ]:
# 這是 completion 風格(蠻多教材仍這樣寫)
user_message = """請分類以下文字是 neutral, negative 或 positive
文字: 我覺得這個披薩實在太好吃啦
情緒:
"""

response = model.generate_content(user_message)
print(response.text)


In [ ]:
# 可改成使用 system instruction 的風格: 把不變的整體指示放在 system instruction
# user prompt 放變動的用戶輸入

# 使用 system instruction 創建模型
model_with_system = genai.GenerativeModel(
    'gemini-1.5-flash',
    system_instruction="請分類以下文字是 neutral, negative 或 positive"
)

user_message = """
文字: 我覺得這個披薩實在太好吃啦
"""

response = model_with_system.generate_content(user_message)
print(response.text)


In [ ]:
# 使用 ChatSession 進行連續對話
model_chat = genai.GenerativeModel('gemini-1.5-flash', 
                                   system_instruction="You are a helpful assistant.")

chat = model_chat.start_chat()

# 第一輪問答
response1 = chat.send_message("誰贏得2013年的世界棒球經典賽冠軍?")
print("第一輪回答:", response1.text)


In [ ]:
# 延續同一個對話的 第二輪問答
response2 = chat.send_message("那2017年呢?")
print("第二輪回答:", response2.text)


In [ ]:
# 延續同一個對話的 第三輪問答
response3 = chat.send_message("美國隊贏過幾次冠軍?")
print("第三輪回答:", response3.text)


In [ ]:
# 如果 第二輪問答時 是問 2018 年
chat_new = model_chat.start_chat()
chat_new.send_message("誰贏得2013年的世界棒球經典賽冠軍?")
response4 = chat_new.send_message("那2018年呢?")
print("2018年回答:", response4.text)


In [ ]:
# 換一種問法 減少幻覺現象
chat_better = model_chat.start_chat()
chat_better.send_message("誰贏得2013年的世界棒球經典賽冠軍?")
response5 = chat_better.send_message("那2018年呢? 如果沒舉辦，請回答沒舉辦")
print("改善後的2018年回答:", response5.text)


In [ ]:
# 出處: https://www.promptingguide.ai/zh/techniques/fewshot
prompt = f"""
請判斷情緒:

input: 這太棒了！
output: Positive

input: 這太糟糕了！
output: Negative

input: 哇，那部電影太棒了！
output: Positive

input: 多麼可怕的節目
output:
"""

response = model.generate_content(prompt)
print(response.text)


In [ ]:
# *在一些較難描述明確指示的任務中，蠻適合用* few-shot 的方式讓模型自己學，例如文字風格、特定的輸出結構(某種schema)

# 沒給範例
prompt = f"""
晶晶體是一種流行於臺灣以中文為基底，夾雜英語不成句的單字或片語的表達方式。特指所使用的英文字多為過於簡單、沒有替換必要者，進而產生有意炫耀雙語能力卻弄巧成拙的效果。

原文: 每位員工都要參加每週電話會議，沒有例外
晶晶體:
"""

response = model.generate_content(prompt)
print(response.text)


In [ ]:
# 給範例讓模型學風格，可以學得更好
prompt = f"""
晶晶體是一種流行於臺灣以中文為基底，夾雜英語不成句的單字或片語的表達方式。特指所使用的英文字多為過於簡單、沒有替換必要者，進而產生有意炫耀雙語能力卻弄巧成拙的效果。
例如:

原文: 我很忙，因為我很有事要做
晶晶體: 我是很busy，因為我很多things要do

原文: 天氣總算放晴，沒有下雨、太陽很大、有點熱、讓我想到以前還是學生時，喜歡在這樣的天氣，吃一球冰淇淋，真的會讓人很高興
晶晶體: 天氣總算放晴，沒有rain、太陽很big、有點hot、讓我想到以前還是student時，喜歡在這樣的天氣，吃一球ice cream，真的會讓人很happy

原文: 每位員工都要參加每週電話會議，沒有例外
晶晶體:
"""

response = model.generate_content(prompt)
print(response.text)


In [ ]:
# 出處: https://promptingguide.azurewebsites.net/techniques/cot
prompt = """
我去市場買了10個蘋果。我給了鄰居2個蘋果，又給修理工2個蘋果。之後，我又去買了5個蘋果，然後吃了1個。我還剩下多少個蘋果？
"""

response = model.generate_content(prompt)
print(response.text)


In [ ]:
# Few-shot CoT - 給一個推理範例，也就是 Chain of Thought (CoT) 思考過程
prompt = """
Q: 我去市場買了6個香蕉，給了朋友3個香蕉，我還剩下多少個?
A:
  1. 我一開始有6個
  2. 給了朋友3個，所以剩下 6-3=3個香蕉
  3. 最後剩下3個香蕉

Q: 我去市場買了10個蘋果。我給了鄰居2個蘋果，又給修理工2個蘋果。之後，我又去買了5個蘋果，然後吃了1個。我還剩下多少個蘋果？
A:
"""

response = model.generate_content(prompt)
print(response.text)


In [ ]:
# Zero-shot CoT (讓模型自己想步驟)
# 標準咒語是 Let's think step by step
prompt = """
我去市場買了10個蘋果。我給了鄰居2個蘋果，又給修理工2個蘋果。之後，我又去買了5個蘋果，然後吃了1個。我還剩下多少個蘋果？
Let's think step by step
"""

response = model.generate_content(prompt)
print(response.text)


In [ ]:
prompt = "請隨機產生三個 user 資料，請用 JSON 格式回傳"

response = model.generate_content(prompt)
print(response.text)


In [ ]:
# Gemini 的 Structured Output 功能
# 文件: https://ai.google.dev/gemini-api/docs/json-mode

# 定義 JSON Schema
response_schema = {
    "type": "object",
    "properties": {
        "name": {
            "type": "string",
            "description": "請用台灣常見姓名"
        },
        "age": {
            "type": "integer",
            "description": "年紀"
        },
        "bio": {
            "type": "string",
            "description": "請用台灣繁體中文"
        },
        "avatar_url": {
            "type": "string",
            "description": "個人圖像，請用真實可以連結的圖片"
        },
        "isSubscriber": {
            "type": "boolean",
            "description": "是否訂閱"
        }
    },
    "required": [
        "name",
        "age", 
        "bio",
        "avatar_url",
        "isSubscriber"
    ]
}

# 使用 response schema 的模型
model_structured = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config=genai.GenerationConfig(
        response_mime_type="application/json",
        response_schema=response_schema
    )
)

response = model_structured.generate_content(prompt)
print(response.text)


In [ ]:
# 使用 Pydantic 定義 schema 更方便
from typing import List
from pydantic import Field, BaseModel, ConfigDict

class User(BaseModel):
    name: str = Field(description="請用台灣常見姓名")
    age: int = Field(description="年紀")
    bio: str = Field(description="請用台灣繁體中文")
    avatar_url: str = Field(description="個人圖像，請用真實可以連結的圖片")
    isSubscriber: bool = Field(description="是否訂閱")

class Users(BaseModel):
    users: list[User]

# 將 Pydantic model 轉換為 Gemini 可用的 schema
def pydantic_to_gemini_schema(pydantic_model):
    schema = pydantic_model.model_json_schema()
    return schema

users_schema = pydantic_to_gemini_schema(Users)
print("Generated Schema:", users_schema)


In [ ]:
# 使用 Pydantic schema 創建模型
model_pydantic = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config=genai.GenerationConfig(
        response_mime_type="application/json",
        response_schema=users_schema
    )
)

response = model_pydantic.generate_content("請隨機產生多個 user 資料")
print(response.text)

# 解析回應到 Pydantic 物件
import json
response_data = json.loads(response.text)
parsed_users = Users(**response_data)
print("\n解析後的第一個用戶:", parsed_users.users[0])


In [ ]:
# Step 1: 從用戶問題中，用 prompt1 來提取出 外部工具的參數
from typing import List
from pydantic import Field, BaseModel

class QueryResult(BaseModel):
    date: str = Field(description="Date in yyyymmdd format. Leave empty if not parsable")
    stock_code: str = Field(description="Taiwan stock code as a 4-digit number. Leave empty if not parsable")

query = "請問2025年4月10號的台積電，股價表現如何?"

# 創建專門用於參數提取的模型
extract_model = genai.GenerativeModel(
    'gemini-1.5-flash',
    system_instruction="Extract from user queries",
    generation_config=genai.GenerationConfig(
        response_mime_type="application/json",
        response_schema=pydantic_to_gemini_schema(QueryResult)
    )
)

response = extract_model.generate_content(query)
extracted_data = json.loads(response.text)
parsed_result = QueryResult(**extracted_data)
print("提取的參數:", parsed_result)


In [ ]:
# Step 2: 執行工具，拿到結果
# API 參考自 https://medium.com/%E5%B7%A5%E7%A8%8B%E9%9A%A8%E5%AF%AB%E7%AD%86%E8%A8%98/5%E7%A8%AE%E6%8A%93%E5%8F%96%E5%8F%B0%E8%82%A1%E6%AD%B7%E5%8F%B2%E8%82%A1%E5%83%B9%E7%9A%84%E6%96%B9%E6%B3%95-766bf2ed9d6

import requests
import json

date = parsed_result.date
stock_code = parsed_result.stock_code
url = 'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=%s&stockNo=%s' % (date, stock_code)

html = requests.get(url)
context = json.loads(html.text)
print("取得的股價資料:", context)


In [ ]:
# Step 3: 用 (prompt2 + 結果) 轉成自然語言回給用戶
prompt = f"""
Based on the provided context, please answer the following question in Traditional Chinese (Taiwan):

Question: <question>{query}</question>

Context: <context>{context}</context>

Instructions:
1. Carefully verify that your answer is supported by the given context.
2. If the question cannot be answered from the provided context, respond with: "抱歉，在提供的資料中找不到相關資訊，無法回答您的問題。"
3. Do not include information that is not present in the context.
4. Ensure your response is written in Traditional Chinese as used in Taiwan.
"""

response = model.generate_content(prompt)
print("最終回答:", response.text)


In [ ]:
# Step 1: 定義工具函式
def web_search(keyword: str) -> str:
    """搜尋最新的資訊"""
    # 模擬搜尋結果
    return f"搜尋關鍵字 '{keyword}' 的結果: 今天台北天氣晴朗，氣溫25度，適合外出活動。"

# Step 2: 定義函數描述給 Gemini
web_search_func = genai.protos.FunctionDeclaration(
    name="web_search",
    description="搜尋最新的資訊",
    parameters=genai.protos.Schema(
        type=genai.protos.Type.OBJECT,
        properties={
            "keyword": genai.protos.Schema(
                type=genai.protos.Type.STRING,
                description="搜尋關鍵字"
            )
        },
        required=["keyword"]
    )
)

# Step 3: 創建包含工具的模型
model_with_tools = genai.GenerativeModel(
    'gemini-1.5-flash',
    tools=[genai.protos.Tool(function_declarations=[web_search_func])]
)

# Step 4: 發送請求
from datetime import datetime
today = datetime.now().strftime("%Y/%m/%d")
response = model_with_tools.generate_content(f"今天 {today} 台北天氣如何?")

print("模型回應:")
print(response.text)
print("\nFunction Calls:")
for part in response.parts:
    if part.function_call:
        print(f"Function: {part.function_call.name}")
        print(f"Arguments: {dict(part.function_call.args)}")


In [ ]:
# Step 5: 處理函數調用並執行
available_functions = {
    "web_search": web_search
}

# 如果有函數調用，執行它們
if any(part.function_call for part in response.parts):
    # 收集函數調用結果
    function_responses = []
    
    for part in response.parts:
        if part.function_call:
            function_name = part.function_call.name
            function_args = dict(part.function_call.args)
            
            print(f"執行函數: {function_name}({function_args})")
            
            # 調用實際函數
            function_to_call = available_functions[function_name]
            function_result = function_to_call(**function_args)
            
            print(f"函數結果: {function_result}")
            
            # 準備函數回應
            function_response = genai.protos.Part(
                function_response=genai.protos.FunctionResponse(
                    name=function_name,
                    response={"result": function_result}
                )
            )
            function_responses.append(function_response)
    
    # Step 6: 將函數結果回傳給模型生成最終回應
    final_response = model_with_tools.generate_content([
        genai.protos.Content(parts=response.parts),  # 原始回應
        genai.protos.Content(parts=function_responses)  # 函數結果
    ])
    
    print("\n最終回應:")
    print(final_response.text)


In [ ]:
# 包成一個完整的 function calling 輔助函數
def run_gemini_with_functions(model, user_message, available_functions):
    """執行帶有函數調用的 Gemini 請求"""
    
    print(f"💬 用戶問題: {user_message}")
    
    # 第一次調用
    response = model.generate_content(user_message)
    
    # 檢查是否有函數調用
    function_calls = [part for part in response.parts if part.function_call]
    
    if function_calls:
        print("🔧 檢測到函數調用")
        
        # 執行所有函數調用
        function_responses = []
        
        for part in response.parts:
            if part.function_call:
                function_name = part.function_call.name
                function_args = dict(part.function_call.args)
                
                print(f"   ⚙️ 執行函數 {function_name} 參數 {function_args}")
                
                # 調用實際函數
                function_to_call = available_functions[function_name]
                function_result = function_to_call(**function_args)
                
                # 準備函數回應
                function_response = genai.protos.Part(
                    function_response=genai.protos.FunctionResponse(
                        name=function_name,
                        response={"result": function_result}
                    )
                )
                function_responses.append(function_response)
        
        # 第二次調用，包含函數結果
        final_response = model.generate_content([
            genai.protos.Content(parts=response.parts),
            genai.protos.Content(parts=function_responses)
        ])
        
        return final_response.text
    else:
        # 沒有函數調用，直接返回
        return response.text

# 測試這個輔助函數
result = run_gemini_with_functions(
    model_with_tools, 
    "今天台北、新竹、高雄的天氣如何？", 
    available_functions
)

print("------")
print("最終結果:", result)


In [ ]:
# 圖像處理範例
# 首先我們需要上傳一個圖像文件到 Gemini

# 方法1: 從 URL 載入圖像 (需要 Pillow)
%pip install Pillow

import PIL.Image
import requests
from io import BytesIO

# 下載一個範例圖像
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
response = requests.get(image_url)
image = PIL.Image.open(BytesIO(response.content))

# 使用 Gemini 分析圖像
multimodal_model = genai.GenerativeModel('gemini-1.5-flash')
response = multimodal_model.generate_content([
    "請詳細描述這張圖片的內容，包括景觀、顏色、氛圍等。請用繁體中文回答。",
    image
])

print("圖像分析結果:")
print(response.text)
